# LibCBM versus CBM-CFS3 Stand level testing

In [6]:
import os, json, math
import numpy as np
import pandas as pd
%matplotlib inline


libCBM related imports

In [7]:
from libcbmwrapper import LibCBMWrapper
import libcbmconfig
import cbmconfig
import cbm_defaults

cbm3 related imports

In [8]:
import cbm3_python_helper
cbm3_python_helper.load_cbm3_python()
import cbm3_python.simulation.projectsimulator
from cbm3_python.cbm3data import sit_helper
from cbm3_python.cbm3data import cbm3_results

In [72]:
db_path = 'C:\dev\cbm_defaults\cbm_defaults.db'
n_steps = 225

age_interval = 5 #required by cbm3
num_age_classes = 20 #required by cbm3

def get_classifier_name(id):
    return str(id)

def growth_curve(x, x_0=65, L=250, k=0.1 ):
    return (x, L/(1+math.exp(-k*(x-x_0))))
    
def create_scenario(id, admin_boundary, eco_boundary, components, events):
    return {
        "id":id,
        "admin_boundary": admin_boundary,
        "eco_boundary": eco_boundary,
        "components": components,
        "events": events
    }

def generate_scenarios(random_seed, num_cases, dbpath, ndigits):
    np.random.seed(random_seed)
    species_ref = cbm_defaults.load_species_reference(dbpath, "en-CA")
    species = [k for k,v in species_ref.items() if len(k)<50 and v["forest_type_id"] in [1,3]] #exclude species names that are too long for the project database schema
    
    spatial_units = cbm_defaults.get_spatial_unit_ids_by_admin_eco_name(dbpath, "en-CA")
    random_spus = np.random.choice([",".join(x) for x in spatial_units.keys()], num_cases)
    
    disturbance_types = cbm_defaults.get_disturbance_type_ids_by_name(dbpath, "en-CA")
    
    cases = []
    for i in range(num_cases):
        num_components = np.random.randint(1,5)
        random_species = np.random.choice(list(species), num_components)
        spu = random_spus[i].split(',')
        components = []
        for c in range(num_components):
            components.append({
                "species": random_species[c],
                "age_volume_pairs": [growth_curve(x) for x in range(0,200,age_interval)]
            })

        num_disturbances = np.random.randint(0,3)
        
        cases.append(create_scenario(
            id = i+1,
            admin_boundary=spu[0],
            eco_boundary=spu[1],
            components = components))
    return cases

In [71]:
generate_scenarios(1, 1, db_path, 2)

#pd.DataFrame({"x": list(range(0, 200 , 5)), "y": [growth_curve(x,65,100,0.1) for x in range(0, 200 , 5)]}).groupby("x").sum().plot()
#list(range(0, 200 , 20))

[{'id': 1,
  'admin_boundary': 'Yukon Territory',
  'eco_boundary': 'Taiga Plains',
  'components': [{'species': 'Alternate-leaf dogwood',
    'age_volume_pairs': [(0, 0.3752955641842479),
     (5, 0.6181557891586936),
     (10, 1.0175344289740318),
     (15, 1.673212731071214),
     (20, 2.746735657648295),
     (25, 4.496552490522889),
     (30, 7.32805768783908),
     (35, 11.856468294391695),
     (40, 18.96454500531089),
     (45, 29.800730505529387),
     (50, 45.60638095158909),
     (55, 67.23535534249878),
     (60, 94.38516719953635),
     (65, 125.0),
     (70, 155.61483280046363),
     (75, 182.7646446575012),
     (80, 204.3936190484109),
     (85, 220.1992694944706),
     (90, 231.03545499468913),
     (95, 238.14353170560832),
     (100, 242.6719423121609),
     (105, 245.5034475094771),
     (110, 247.2532643423517),
     (115, 248.32678726892883),
     (120, 248.98246557102598),
     (125, 249.38184421084134),
     (130, 249.62470443581574),
     (135, 249.772237201399

In [ ]:

standard_import_tool_plugin_path=sit_helper.load_standard_import_tool_plugin()

toolbox_path = r"C:\Program Files (x86)\Operational-Scale CBM-CFS3"

#there is a bug fix in this version of cbm/makelist for growth incrment blips
cbm_exe_path = r"M:\CBM Tools and Development\Builds\CBMBuilds\20190530_growth_increment_fix"

cbm3_project_dir = os.path.join(toolbox_path, "Projects", "libcbm_stand_level_testing")
cbm3_project_path = os.path.join(cbm3_project_dir, "libcbm_stand_level_testing.mdb")
cbm3_results_db_path = os.path.join(cbm3_project_dir, "libcbm_stand_level_testing_results.mdb")
config_save_path = os.path.join(cbm3_project_dir, "libcbm_stand_level_testing.json")
sit_config = sit_helper.SITConfig(
    imported_project_path=cbm3_project_path,
    initialize_mapping=True
)
sit_config.data_config(
    age_class_size=age_interval,
    num_age_classes=num_age_classes,
    classifiers=["admin", "eco", "identifier", "species"])
sit_config.set_admin_eco_mapping("admin","eco")
sit_config.set_species_classifier("species")